# Data Migration Tracking - Optimized

### Description
To track data migration status and filings done after migration - Optimized version with fewer queries and dataframes

#### Install Dependencies

In [17]:
# Run this cell if you haven't installed these packages
!pip install pandas openpyxl sqlalchemy numpy psycopg2-binary python-dotenv


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Setup and COnfiguration

In [18]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from typing import List, Any
from openpyxl.styles import Font, Alignment
from datetime import datetime

load_dotenv()

# Configuration
GROUP_TABLE_FOLDER = os.getenv('GROUP_TABLE_FOLDER')
GROUP_TABLE_FILE_NAME = os.getenv('GROUP_TABLE_FILE_NAME')
OUTPUT_FOLDER = os.getenv('OUTPUT_FOLDER')
COLUMN_FOR_CORP_NUM = os.getenv('COLUMN_FOR_CORP_NUM')

# Colin Extracts DB
COLIN_EXTRACT_DB = os.getenv('COLIN_EXTRACT_DB')
CE_HOST_URL = os.getenv('CE_HOST_URL')
CE_USERNAME = os.getenv('CE_USERNAME')
CE_PASSWORD = os.getenv('CE_PASSWORD')
CE_PORT = os.getenv('CE_PORT')

# Lear DB
LEAR_DB = os.getenv('LEAR_DB')
LEAR_HOST_URL = os.getenv('LEAR_HOST_URL')
LEAR_USERNAME = os.getenv('LEAR_USERNAME')
LEAR_PASSWORD = os.getenv('LEAR_PASSWORD')
LEAR_PORT = os.getenv('LEAR_PORT')

# Column mapping
COLUMN_NAMES = {
    "corp_num": "Incorporation Number",
    "corp_name": "Company Name",
    "corp_type": "Type",
    "email": "Admin Email",
    "status": "Migration Status",
    "date": "Migrated Date",
    "filings": "Filings Done",
    "filing_date": "Last Filing Date"
}

PRINT_DIVIDER = "=" * 50


In [19]:
def get_db_connection_string(host_address: str, database: str, user_name: str, db_password: str, port: str = "5432") -> str:
    """Create db connection string."""
    return f"postgresql://{user_name}:{db_password}@{host_address}:{port}/{database}"


### Read Excel file and extract corp_nums

In [20]:
try:
    full_group_table_path = f"{GROUP_TABLE_FOLDER}/{GROUP_TABLE_FILE_NAME}"
    corp_nums_df = pd.read_excel(full_group_table_path, sheet_name="Sheet1", usecols=[COLUMN_FOR_CORP_NUM])
    corp_nums_df = corp_nums_df.sort_values(COLUMN_FOR_CORP_NUM)
    
    corp_num_column_values = corp_nums_df[COLUMN_FOR_CORP_NUM].dropna().tolist()
    
    print(f"Found {len(corp_num_column_values)} corps in the group table")
    print(f"All corps: {corp_num_column_values}")
    print(PRINT_DIVIDER)

except Exception as e:
    print(f"Error reading Excel file: {e}")
    corp_num_column_values = []


Found 64 corps in the group table
All corps: ['BC0754828', 'BC0769801', 'BC0910591', 'BC0934777', 'BC0934782', 'BC0971192', 'BC0988623', 'BC1033896', 'BC1034551', 'BC1041519', 'BC1049046', 'BC1055974', 'BC1065213', 'BC1072742', 'BC1080101', 'BC1082247', 'BC1090168', 'BC1105588', 'BC1113730', 'BC1139090', 'BC1140525', 'BC1161928', 'BC1169589', 'BC1172657', 'BC1173897', 'BC1179877', 'BC1180203', 'BC1185476', 'BC1196188', 'BC1201211', 'BC1208807', 'BC1211791', 'BC1221371', 'BC1228520', 'BC1245585', 'BC1246637', 'BC1249698', 'BC1250621', 'BC1256884', 'BC1292656', 'BC1297308', 'BC1302343', 'BC1308092', 'BC1341547', 'BC1341825', 'BC1344052', 'BC1361825', 'BC1363286', 'BC1387185', 'BC1395304', 'BC1400407', 'BC1411915', 'BC1414068', 'BC1417733', 'BC1428110', 'BC1440160', 'BC1475529', 'BC1481042', 'BC1483392', 'BC1484094', 'BC1484169', 'BC1484174', 'BC1507435', 'BC1507445']


### Connect to Colin Extracts db and get all Data

In [21]:
try:
    colin_extracts_connection_string = get_db_connection_string(
        CE_HOST_URL, COLIN_EXTRACT_DB, CE_USERNAME, CE_PASSWORD, CE_PORT
    )
    colin_extracts_engine = create_engine(colin_extracts_connection_string)
    
    # Test connection
    with colin_extracts_engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("Colin Extracts database connection successful")

except Exception as e:
    print(f"Connection to Colin Extracts failed: {e}")
    colin_extracts_engine = None

print(PRINT_DIVIDER)


Colin Extracts database connection successful


In [22]:
# Get corp_name, corp_type, admin_email, migration_status, migration_date from Colin Extracts
if colin_extracts_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"corp_num IN ('{values_str}')"

        # Single comprehensive query joining all needed tables
        combined_query = f"""
        SELECT 
            corp.corp_num as "{COLUMN_NAMES['corp_num']}",
            cn.corp_name as "{COLUMN_NAMES['corp_name']}",
            corp.corp_type_cd as "{COLUMN_NAMES['corp_type']}",
            corp.admin_email as "{COLUMN_NAMES['email']}",
            CASE 
                WHEN cp.processed_status = 'COMPLETED' THEN 'Migrated'
                ELSE 'Pending'
            END as "{COLUMN_NAMES['status']}",
            cp.create_date::date as "{COLUMN_NAMES['date']}"
        FROM public.corporation corp
        LEFT JOIN public.corp_name cn ON corp.corp_num = cn.corp_num 
            AND cn.corp_name_typ_cd IN ('CO', 'NB') 
            AND cn.end_event_id IS NULL
        LEFT JOIN public.corp_processing cp ON corp.corp_num = cp.corp_num
        WHERE corp.{in_clause}
        ORDER BY corp.corp_num
        """
        
        # Execute combined query
        tracking_df = pd.read_sql(combined_query, colin_extracts_engine)
        
        # Convert date column
        tracking_df[COLUMN_NAMES['date']] = pd.to_datetime(tracking_df[COLUMN_NAMES['date']])
        
        print(f"Retrieved data for {len(tracking_df)} corporations from Colin Extracts")
        display(tracking_df.head())
        print(PRINT_DIVIDER)
        
    except Exception as e:
        print(f"Failed to execute combined query: {e}")
        tracking_df = pd.DataFrame()


Retrieved data for 64 corporations from Colin Extracts


,Incorporation Number,Company Name,Type,Admin Email,Migration Status,Migrated Date
0,BC0754828,CLINSCAPE CONSULTING INC.,BC,vancorp@bennettjones.com,Migrated,2025-04-16
1,BC0769801,FREE THE GOLDFISH COACHING INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09
2,BC0910591,MOKO PROPERTY GROUP INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16


### Connect to LEAR DB and get post-migration filings data

In [23]:
# Connect to LEAR DB
try:
    lear_connection_string = get_db_connection_string(
        LEAR_HOST_URL, LEAR_DB, LEAR_USERNAME, LEAR_PASSWORD, LEAR_PORT
    )
    lear_engine = create_engine(lear_connection_string)
    
    # Test connection
    with lear_engine.connect() as lear_conn:
        lear_conn.execute(text("SELECT 1"))
    print("LEAR database connection successful")

except Exception as e:
    print(f"Connection to LEAR failed: {e}")
    lear_engine = None

print(PRINT_DIVIDER)


LEAR database connection successful


In [24]:
# Get post-migration filings data (filing types and last filing date) from LEAR
def convert_filings_to_title_case(filings_string: str) -> str:
        """Convert camelCase to Title Case (e.g., 'annualReport' -> 'Annual Report')"""
        import re
        result = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', filings_string)
        return result.title()


if lear_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"b.identifier IN ('{values_str}')"
        
        # Combined query to get business IDs and filings in one go
        lear_combined_query = f"""
        WITH filing_summary AS (
            SELECT 
                b.identifier,
                STRING_AGG(
                    DISTINCT filing_type,
                    ', '
                ) as filings_done,
                MAX(f.filing_date)::date as last_filing_date
            FROM public.businesses b
            LEFT JOIN public.filings f ON b.id = f.business_id 
            AND f.source = 'LEAR'
            AND f.status = 'COMPLETED'
            WHERE {in_clause}
            GROUP BY b.identifier
        )
        SELECT 
            identifier as "{COLUMN_NAMES['corp_num']}",
            COALESCE(filings_done, '') as "{COLUMN_NAMES['filings']}",
            COALESCE(last_filing_date::text, '') as "{COLUMN_NAMES['filing_date']}"
        FROM filing_summary
        ORDER BY identifier
        """
        
        lear_filings_df = pd.read_sql(lear_combined_query, lear_engine)
        
        print(f"Retrieved LEAR filings data for {len(lear_filings_df)} corporations")
        if not lear_filings_df.empty:
            lear_filings_df[COLUMN_NAMES['filings']] = lear_filings_df[COLUMN_NAMES['filings']].apply(convert_filings_to_title_case)
            display(lear_filings_df)

    except Exception as e:
        print(f"Error getting LEAR filings data: {e}")
        lear_filings_df = pd.DataFrame()
    
print(PRINT_DIVIDER)


Retrieved LEAR filings data for 64 corporations


,Incorporation Number,Filings Done,Last Filing Date
0,BC0754828,,
1,BC0769801,,
2,BC0910591,,
3,BC0934777,,
4,BC0934782,,
...,...,...,...
59,BC1484094,,
60,BC1484169,,
61,BC1484174,"Annual Report, Change Of Directors",2025-06-24
62,BC1507435,,


### Combine All Data and Save Final Result

In [25]:
def generate_unique_filename(original_path: str) -> str:
    """Generate unique filename with date and incremental number if file exists."""
    directory = os.path.dirname(original_path)
    filename = os.path.basename(original_path)
    name, ext = os.path.splitext(filename)
    
    today = datetime.now().strftime('%Y%m%d')
    new_filename = f"{name}_{today}{ext}"
    new_path = os.path.join(directory, new_filename)
    
    if not os.path.exists(new_path):
        return new_path
    
    counter = 2
    while counter <= 999:
        incremental_filename = f"{name}_{today}_{counter:02d}{ext}"
        incremental_path = os.path.join(directory, incremental_filename)
        
        if not os.path.exists(incremental_path):
            return incremental_path
        
        counter += 1
    
    raise Exception("Too many files with the same name pattern")


def format_and_save_excel(df: pd.DataFrame, file_save_path: str = 'output.xlsx', sheet_name: str = 'Sheet1') -> None:
    """Save to Excel file with basic freeze and alignment."""
    file_save_path = generate_unique_filename(file_save_path)

    with pd.ExcelWriter(file_save_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]
        
        # Freeze the 1st row
        worksheet.freeze_panes = 'A2'
        
        # Make header bold and left-align all cells
        for cell in worksheet[1]:
            cell.font = Font(bold=True)
            cell.alignment = Alignment(horizontal='left')
        
        for row in worksheet.iter_rows(min_row=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='left')
        
        # Adjust column widths
        for column in worksheet.columns:
            max_length = max(len(str(cell.value)) for cell in column if cell.value)
            worksheet.column_dimensions[column[0].column_letter].width = max_length + 2
    
    print(f"DataFrame saved to {file_save_path} with frozen header and left alignment")


In [26]:
try:
    if not tracking_df.empty:
        # Merge with LEAR filings data
        if not lear_filings_df.empty:
            final_df = tracking_df.merge(
                lear_filings_df,
                on=COLUMN_NAMES['corp_num'],
                how='left'
            )
        else:
            # If no LEAR data, add empty columns
            final_df = tracking_df.copy()
            final_df[COLUMN_NAMES['filings']] = ''
            final_df[COLUMN_NAMES['filing_date']] = ''
        
        # Convert date column to string for Excel export
        final_df[COLUMN_NAMES['date']] = final_df[COLUMN_NAMES['date']].astype(str)
        
        print(f"Final dataset contains {len(final_df)} corporations")
        print(f"Corporations with filings: {len(final_df[final_df[COLUMN_NAMES['filings']] != ''])}")
        
        # Display sample of final data
        with pd.option_context('display.max_rows', None):  # Adjust None to a number when dataset is big
            display(final_df)
        
        # Save to Excel
        output_path = f"{OUTPUT_FOLDER}/migration_tracking_result.xlsx"
        format_and_save_excel(final_df, output_path)
        
        print(PRINT_DIVIDER)
        print("Processing completed successfully!")
        
    else:
        print("No data retrieved from Colin Extracts database")
        
except Exception as e:
    print(f"Error in final data processing: {e}")


Final dataset contains 64 corporations
Corporations with filings: 3


,Incorporation Number,Company Name,Type,Admin Email,Migration Status,Migrated Date,Filings Done,Last Filing Date
0,BC0754828,CLINSCAPE CONSULTING INC.,BC,vancorp@bennettjones.com,Migrated,2025-04-16,,
1,BC0769801,FREE THE GOLDFISH COACHING INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
2,BC0910591,MOKO PROPERTY GROUP INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16,,
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16,,
5,BC0971192,BOWERY FUNDING ULC,ULC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
6,BC0988623,NHI DENIS PERSONAL REAL ESTATE CORPORATION,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
7,BC1033896,1033896 B.C. LTD.,BC,vancorp@bennettjones.com,Migrated,2025-06-13,,
8,BC1034551,CHRISTIAN P. GAUTHIER LAW CORPORATION,BC,VanCorp@bennettjones.com,Migrated,2025-06-13,,
9,BC1041519,FRIND ENTERPRISES LTD.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,


DataFrame saved to /mnt/c/Users/EASPAN/Downloads/migration_tracking_result_20250626.xlsx with frozen header and left alignment
Processing completed successfully!
